In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("filtered_data/data.csv")
df.head()

,moves,explanation,instruction
0,e4 e6 d4 b6 e5 Bb7 Nf3 h6 Bd3 g5 O-O g4 Nfd2 h...,### Game Analysis\n\n**Move Pair 1:** \n- **Wh...,"Assume you are a chess master, explain the str..."
1,e4 e6 d4 d5 Nd2 dxe4 Nxe4 Nf6 Bd3 Nxe4 Bxe4 Nc...,"### Game Analysis\n1. **White: e4, Black: e6**...","Assume you are a chess master, explain the str..."
2,e4 e6 b3 d5 f3 Nc6 Bb2 dxe4 fxe4 Qh4+ Ke2 Qxe4...,"### Game Analysis\n\n1. **White: e4, Black: e6...","Assume you are a chess master, explain the str..."
3,e4 e5 Nf3 Nc6 d4 Bb4+ c3 Bd6 Bb5 Qf6 O-O exd4 ...,"### Game Analysis:\n\n1. **White: e4, Black: e...","Assume you are a chess master, explain the str..."
4,d4 Nf6 c4 e6 Nf3 d5 c5 b6 b4 a5 Ba3 axb4 Bxb4 ...,"### Game Analysis\n\n1. **d4, Nf6**\n - **Wh...","Assume you are a chess master, explain the str..."


In [11]:
df_exp = df.iloc[:100]
df_exp.shape

(100, 3)

In [49]:
def fix_text_winner(text:str):
    to_replace = ["*", ":","\n", "-"]
    target = text[:]
    for c in to_replace:
        target = target.replace(str(c), "")
    return target.strip()

fix_text_winner(text.split("Winner")[len(text.split("Winner"))-1])

"White wins. White's final move, pushing the e6 pawn, creates an overwhelming threat that Black cannot counter effectively. The black king's precarious position without castling and the material disadvantage led to White achieving a winning position."

In [46]:
text = df.iloc[0]["explanation"]
text.split("Winner")[len(text.split("Winner"))-1]

":**\n- **White** wins. White's final move, pushing the e6 pawn, creates an overwhelming threat that Black cannot counter effectively. The black king's precarious position without castling and the material disadvantage led to White achieving a winning position."

In [14]:
df.iloc[0]["instruction"]

'Assume you are a chess master, explain the strategy used by each player based on the provided chess moves.'

In [10]:
df.columns

Index(['moves', 'explanation', 'instruction'], dtype='object')

## taking 100 random rows for winner task

In [51]:
df_win_temp = df.sample(100, random_state=42)

In [60]:
def generate_winner_task(df):
    instruction = []
    moves  = []
    explanation = []
    for i in range(df.shape[0]):
        instruction.append("Assume you are a chess master. Who do you think will win the game based on the provided chess moves in Algebraic Notation.")
        moves.append(df.iloc[i]["moves"])
        target = df.iloc[i]["explanation"]
        target = target.split("Winner")
        output = target[len(target)-1]
        explanation.append(fix_text_winner(output))

    return list(zip(moves, explanation, instruction))

In [61]:
df_win = pd.DataFrame(generate_winner_task(df_win_temp), columns=["moves", "explanation", "instruction"])

In [62]:
df_win.head()

,moves,explanation,instruction
0,e4 c5 Nf3 d6 Bc4 Nf6 d3 Bg4 Nbd2 Nc6 c3 e5 Qb3...,"By the analysis of the final position, White s...",Assume you are a chess master. Who do you thin...
1,d4 Nf6 c4 g6 Nc3 Bg7 Nf3 d6 Bg5 O-O e3 Nfd7 h3...,"The final move, Be5+, puts heavy pressure on B...",Assume you are a chess master. Who do you thin...
2,e4 c5 f4 d6 Nf3 Nf6 d3 e5 fxe5 dxe5 Nxe5 Be6 B...,White. The decisive sequence at the end forced...,Assume you are a chess master. Who do you thin...
3,e4 e5 Nf3 Nc6 d4 exd4 Nxd4 Bc5 Nb3 Bb6 a4 Qf6 ...,White won the game due to superior positioning...,Assume you are a chess master. Who do you thin...
4,d4 c6 Nf3 d5 Bf4 Bg4 e3 Nd7 Nbd2 f6 h3 Bh5 c4 ...,"White demonstrates superior strategic play, le...",Assume you are a chess master. Who do you thin...


In [63]:
df_win.iloc[4]["explanation"]

'White demonstrates superior strategic play, leveraging tactical opportunities and decisive sacrifice to expose Black’s king and gain the winning position.'

# predict next move task

In [71]:
df_prog = pd.read_csv("../data/progressive_move_data_chunks_5000/data_00.csv")
df_prog = df_prog.sample(100, random_state=42)
df_prog.drop("opening_name", axis = 1, inplace=True)
df_prog.columns = ["moves", "explanation"]
df_prog

,moves,explanation
361,e4 d5,exd5
73,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,Bxh5
374,e4 d5 exd5 Qxd5 Nc3 Qe5+ Be2 Na6 d4 Qf5 Bxa6 b...,Bb7
155,e4 e5 Nf3 d6,d4
104,d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...,Nb5
...,...,...
347,e4 e5 Bc4 Nc6 Nf3 Nd4 d3 Nxf3+ Qxf3 Nf6 h3 Bc5...,Nd3
86,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,Kb4
75,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,Rxh7
438,e4 d5 exd5 Qxd5 Nc3 Qe5+ Be2 Na6 d4 Qf5 Bxa6 b...,Rf4


In [72]:
df_prog["instruction"] = "Assume you are a chess master. Suggest the next move based on the provided ches moves in Algebraic notation"

In [73]:
df_prog.iloc[0]

moves                                                      e4 d5
explanation                                                 exd5
instruction    Assume you are a chess master. Suggest the nex...
Name: 361, dtype: object

In [74]:
df_exp.columns, df_win.columns, df_prog.columns

(Index(['moves', 'explanation', 'instruction'], dtype='object'),
 Index(['moves', 'explanation', 'instruction'], dtype='object'),
 Index(['moves', 'explanation', 'instruction'], dtype='object'))

In [78]:
df = pd.concat([df_exp, df_win, df_prog], ignore_index=True)

In [79]:
df

,moves,explanation,instruction
0,e4 e6 d4 b6 e5 Bb7 Nf3 h6 Bd3 g5 O-O g4 Nfd2 h...,### Game Analysis\n\n**Move Pair 1:** \n- **Wh...,"Assume you are a chess master, explain the str..."
1,e4 e6 d4 d5 Nd2 dxe4 Nxe4 Nf6 Bd3 Nxe4 Bxe4 Nc...,"### Game Analysis\n1. **White: e4, Black: e6**...","Assume you are a chess master, explain the str..."
2,e4 e6 b3 d5 f3 Nc6 Bb2 dxe4 fxe4 Qh4+ Ke2 Qxe4...,"### Game Analysis\n\n1. **White: e4, Black: e6...","Assume you are a chess master, explain the str..."
3,e4 e5 Nf3 Nc6 d4 Bb4+ c3 Bd6 Bb5 Qf6 O-O exd4 ...,"### Game Analysis:\n\n1. **White: e4, Black: e...","Assume you are a chess master, explain the str..."
4,d4 Nf6 c4 e6 Nf3 d5 c5 b6 b4 a5 Ba3 axb4 Bxb4 ...,"### Game Analysis\n\n1. **d4, Nf6**\n - **Wh...","Assume you are a chess master, explain the str..."
...,...,...,...
295,e4 e5 Bc4 Nc6 Nf3 Nd4 d3 Nxf3+ Qxf3 Nf6 h3 Bc5...,Nd3,Assume you are a chess master. Suggest the nex...
296,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,Kb4,Assume you are a chess master. Suggest the nex...
297,e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...,Rxh7,Assume you are a chess master. Suggest the nex...
298,e4 d5 exd5 Qxd5 Nc3 Qe5+ Be2 Na6 d4 Qf5 Bxa6 b...,Rf4,Assume you are a chess master. Suggest the nex...


In [81]:
df["instruction"].unique()

array(['Assume you are a chess master, explain the strategy used by each player based on the provided chess moves.',
       'Assume you are a chess master. Who do you think will win the game based on the provided chess moves in Algebraic Notation.',
       'Assume you are a chess master. Suggest the next move based on the provided ches moves in Algebraic notation'],
      dtype=object)

## shuffle

In [82]:
df = df.sample(frac = 1).reset_index(drop = True)
df

,moves,explanation,instruction
0,e4 e6 d4 d5 Nd2 c5 exd5 exd5 dxc5 Bxc5 Nb3 Bb6...,"White. The game dynamics favored White, who ef...",Assume you are a chess master. Who do you thin...
1,e4 d5 exd5 Qxd5 Nc3 Qe5+ Be2 Na6 d4 Qf5 Bxa6 b...,Qg7#,Assume you are a chess master. Suggest the nex...
2,e4 d5 exd5 Qxd5 Nc3 Qe5+ Be2 Na6 d4 Qf5 Bxa6 b...,Rd2+,Assume you are a chess master. Suggest the nex...
3,d4 d5 Nf3 c5 c3 c4 Bf4 Nc6 e3 e6 Nbd2 Bd6 Bg3 ...,"White, achieved through superior tactical expl...",Assume you are a chess master. Who do you thin...
4,e4 e5 Bc4 Nf6 Nc3 Bc5 Nf3 O-O O-O d6 d4 exd4 N...,"ReasonWhite emerged as the winner, largely due...",Assume you are a chess master. Who do you thin...
...,...,...,...
295,e4 d6 Nf3 Nf6 Nc3 g6 d4 Bg7 Be3 O-O Be2 Nfd7 Q...,"### Game Analysis\n\n#### Move 1: White e4, Bl...","Assume you are a chess master, explain the str..."
296,Nf3 c6 d4 Nf6 c4 d5 Nc3 g6 Bg5 Bg7 e3 O-O Qb3 ...,"### Game Analysis\n1. **White: Nf3, Black: c6*...","Assume you are a chess master, explain the str..."
297,e4 b6 d4 Bb7 Nc3 e6 a3 Nf6 Bd3 Be7 Nge2 d5 e5 ...,The game analysis stops before we can definiti...,Assume you are a chess master. Who do you thin...
298,e4 c5 c3 Nc6 Nf3 e6 d4 cxd4 Nxd4 a6 Nxc6 bxc6 ...,Black Reason Black effectively countered White...,Assume you are a chess master. Who do you thin...


In [85]:
df.to_csv("filtered_data/diverse_data.csv", index = False)